# Calculating the best route among several places.

This implementation demonstrates the usage of [Genetic Algorithm](https://en.wikipedia.org/wiki/Genetic_algorithm) to find the best route through a pre-defined set of places in a [map](https://www.google.com/maps/d/edit?mid=zeE3CQjyMLjM.kVO3Qd7EYm6Y).

**Configuration**

In [1]:
# File paths
kmlMapFilePath = 'map.kml'
waypointsFilePath = 'waypoints.tsv'
outputHtmlFilePath = 'visualize_map.html'

# Google Maps API
googleMapsApiKey = 'your-api-key-here'
directionsMode = 'walking' # Other options: 'driving', 'bicycling'
mapId = 'zeE3CQjyMLjM.kVO3Qd7EYm6Y'

# Genetic Algorithm tuning
generationCount = 5000
populationSize = 100
maxPointMutationsPerOffspring = 3
shuffleMutationsPerOffsprint = 7

**Loading places from Google Maps MyMaps**

In [2]:
from urllib import request

mapUrl = 'https://www.google.com/maps/d/u/0/kml?mid='+mapId+'&output=kml&forcekml=1&cid=mp&cv=z7FSTUgJGII.en.'

with request.urlopen(mapUrl) as urlfile:
    localfile = open(kmlMapFilePath, 'wb')
    localfile.write(urlfile.read())
    localfile.close()

import xml.sax, xml.sax.handler
class PlacemarkHandler(xml.sax.handler.ContentHandler):
    def __init__(self):
        self.inName = False # handle XML parser events
        self.inPlacemark = False
        self.mapping = {} 
        self.buffer = ""
        self.name_tag = ""
        
    def startElement(self, name, attributes):
        if name == "Placemark": # on start Placemark tag
            self.inPlacemark = True
            self.buffer = "" 
        if self.inPlacemark:
            if name == "name": # on start title tag
                self.inName = True # save name text to follow
            
    def characters(self, data):
        if self.inPlacemark: # on text within tag
            self.buffer += data # save text if in title
            
    def endElement(self, name):
        self.buffer = self.buffer.strip('\n\t')
        
        if name == "Placemark":
            self.inPlacemark = False
            self.name_tag = "" #clear current name
        
        elif name == "name" and self.inPlacemark:
            self.inName = False # on end title tag            
            self.name_tag = self.buffer.strip()
            self.mapping[self.name_tag] = {}
        elif self.inPlacemark:
            if name in self.mapping[self.name_tag]:
                self.mapping[self.name_tag][name] += self.buffer
            else:
                self.mapping[self.name_tag][name] = self.buffer
        self.buffer = ""
        
kml = open(kmlMapFilePath, 'r')

parser = xml.sax.make_parser()
handler = PlacemarkHandler()
parser.setContentHandler(handler)
parser.parse(kml)
kml.close()

places = {}
placeslist = []

for key in handler.mapping:
    places[handler.mapping[key]['coordinates']] = key
    coordinate = handler.mapping[key]['coordinates'].rstrip(',0.0')
    placeslist.append(coordinate.split(',')[1]+','+coordinate.split(',')[0])

print('%d places found.' % len(placeslist))
print(placeslist)

17 places found.
['-22.961238400000003,-43.2131231', '-22.9864665,-43.2068467', '-22.9944913,-43.2348222', '-22.91314,-43.1799603', '-22.9457624,-43.197282', '-23.0007675,-43.2727164', '-22.9966937,-43.2459426', '-22.9119442,-43.196649', '-22.913406800000004,-43.2275963', '-22.974258000000003,-43.2011819', '-22.9466961,-43.2177901', '-22.9710823,-43.2170445', '-22.9885307,-43.1925237', '-22.9687066,-43.1794453', '-22.95539,-43.167032', '-22.9663481,-43.2200244', '-22.9871875,-43.2227147']


**Getting distances from Google Maps API and saving waypoints file with the results**

Obs.: In case the waypoints file already exists, it's kept, and no calls are made, to prevent from running out of free API calls (which basically means a 24-hour ban)

In [4]:
import googlemaps
from itertools import combinations
import os

from IPython.display import clear_output

if os.path.exists(waypointsFilePath):
    print('Waypoint file already existed.')
else:
    
    gmaps = googlemaps.Client(key=googleMapsApiKey)

    waypoint_distances = {}
    waypoint_durations = {}

    measurements = 1

    route = None
    
    try:
        for (waypoint1, waypoint2) in combinations(placeslist, 2):
            route = gmaps.distance_matrix(origins=[waypoint1],
                                          destinations=[waypoint2],
                                          mode="walking", # Change this to "walking" for walking directions,
                                                          # "bicycling" for biking directions, etc.
                                          language="English",
                                          units="metric")

            # "distance" is in meters
            distance = route["rows"][0]["elements"][0]["distance"]["value"]

            # "duration" is in seconds
            duration = route["rows"][0]["elements"][0]["duration"]["value"]

            waypoint_distances[frozenset([waypoint1, waypoint2])] = distance
            waypoint_durations[frozenset([waypoint1, waypoint2])] = duration

            print(str(measurements) + ' - From '+waypoint1+' to '+waypoint2+' = '+str(distance))
            measurements += 1

    except Exception as e:
        print("Error with finding the route between %s and %s." % (waypoint1, waypoint2))
        print(route)
    
    with open(waypointsFilePath, 'w') as out_file:
        out_file.write("\t".join(["waypoint1",
                                  "waypoint2",
                                  "distance_m",
                                  "duration_s"]))

        for (waypoint1, waypoint2) in waypoint_distances.keys():
            out_file.write("\n" +
                           "\t".join([waypoint1,
                                      waypoint2,
                                      str(waypoint_distances[frozenset([waypoint1, waypoint2])]),
                                      str(waypoint_durations[frozenset([waypoint1, waypoint2])])]))
    
    print('Total of ' + len(waypoint_distances) + 'Requests completed')

Waypoint file already existed.


**Reading waypoints file**

In [5]:
import pandas as pd
import numpy as np

waypoint_distances = {}
waypoint_durations = {}
placeslist = set()

waypoint_data = pd.read_csv(waypointsFilePath, sep="\t")

for i, row in waypoint_data.iterrows():
    waypoint_distances[frozenset([row.waypoint1, row.waypoint2])] = row.distance_m
    waypoint_durations[frozenset([row.waypoint1, row.waypoint2])] = row.duration_s
    placeslist.update([row.waypoint1, row.waypoint2])
    
print('Waypoints file read. %d distances found' % len(waypoint_distances))

Waypoints file read. 136 distances found


The Genetic Algorithm itself

In a few words, it basically works like this:

1. Generating random routes, in a quantity of the configured population size
2. Getting 10% of the best (shortest) routes (thus reducing the population drastically)
3. Performing random mutations (thus generating new routes, increasing back the population)
4. Repeating steps 2 and 3 for the configured amount of generations
5. Returning the top 1 shortest route of the population in the final generation

In [6]:
import random

def compute_fitness(solution):
    """
        This function returns the total distance traveled on the current road trip.
        
        The genetic algorithm will favor road trips that have shorter
        total distances traveled.
    """
    
    solution_fitness = 0.0
    
    for index in range(len(solution)):
        waypoint1 = solution[index - 1]
        waypoint2 = solution[index]
        solution_fitness += waypoint_distances[frozenset([waypoint1, waypoint2])]
        
    return solution_fitness

def generate_random_agent():
    """
        Creates a random road trip from the waypoints.
    """
    
    new_random_agent = list(placeslist)
    random.shuffle(new_random_agent)
    return tuple(new_random_agent)

def mutate_agent(agent_genome, max_mutations=3):
    """
        Applies 1 - `max_mutations` point mutations to the given road trip.
        
        A point mutation swaps the order of two waypoints in the road trip.
    """
    
    agent_genome = list(agent_genome)
    num_mutations = random.randint(1, max_mutations)
    
    for mutation in range(num_mutations):
        swap_index1 = random.randint(0, len(agent_genome) - 1)
        swap_index2 = swap_index1

        while swap_index1 == swap_index2:
            swap_index2 = random.randint(0, len(agent_genome) - 1)

        agent_genome[swap_index1], agent_genome[swap_index2] = agent_genome[swap_index2], agent_genome[swap_index1]
            
    return tuple(agent_genome)

def shuffle_mutation(agent_genome):
    """
        Applies a single shuffle mutation to the given road trip.
        
        A shuffle mutation takes a random sub-section of the road trip
        and moves it to another location in the road trip.
    """
    
    agent_genome = list(agent_genome)
    
    start_index = random.randint(0, len(agent_genome) - 1)
    length = random.randint(2, 20)
    
    genome_subset = agent_genome[start_index:start_index + length]
    agent_genome = agent_genome[:start_index] + agent_genome[start_index + length:]
    
    insert_index = random.randint(0, len(agent_genome) + len(genome_subset) - 1)
    agent_genome = agent_genome[:insert_index] + genome_subset + agent_genome[insert_index:]
    
    return tuple(agent_genome)

def generate_random_population(pop_size):
    """
        Generates a list with `pop_size` number of random road trips.
    """
    
    random_population = []
    for agent in range(pop_size):
        random_population.append(generate_random_agent())
    return random_population


def run_genetic_algorithm(generations=5000, population_size=100):
    """
        The core of the Genetic Algorithm.
        
        `generations` and `population_size` must be a multiple of 10.
    """
    
    population_subset_size = int(population_size / 10.)
    generations_10pct = int(generations / 10.)
    
    # Create a random population of `population_size` number of solutions.
    population = generate_random_population(population_size)

    # For `generations` number of repetitions...
    for generation in range(generations):
        # Compute the fitness of the entire current population
        population_fitness = {}

        for agent_genome in population:
            if agent_genome in population_fitness:
                continue

            population_fitness[agent_genome] = compute_fitness(agent_genome)

        # Take the top 10% shortest road trips and produce offspring each from them
        new_population = []
        for rank, agent_genome in enumerate(sorted(population_fitness,
                                                   key=population_fitness.get)[:population_subset_size]):
            
            if (generation % generations_10pct == 0 or generation == generations - 1) and rank == 0:
                print("Generation %d best: %d | Unique genomes: %d" % (generation,
                                                                       population_fitness[agent_genome],
                                                                       len(population_fitness)))
                
                best = agent_genome

            # Create 1 exact copy of each of the top road trips
            new_population.append(agent_genome)

            # Create 2 offspring with 1-X (default = 3) point mutations
            for offspring in range(2):
                new_population.append(mutate_agent(agent_genome, maxPointMutationsPerOffspring))
                
            # Create 7 offspring with a single shuffle mutation
            for offspring in range(shuffleMutationsPerOffsprint):
                new_population.append(shuffle_mutation(agent_genome))

        # Replace the old population with the new population of offspring 
        for i in range(len(population))[::-1]:
            del population[i]

        population = new_population
    
    print('------\nCompleted. Best route found:')
    print(best)
        
    return best

result = run_genetic_algorithm(generations=generationCount, population_size=populationSize)

Generation 0 best: 115960 | Unique genomes: 100
Generation 500 best: 57456 | Unique genomes: 81
Generation 1000 best: 57456 | Unique genomes: 85
Generation 1500 best: 57456 | Unique genomes: 74
Generation 2000 best: 57456 | Unique genomes: 75
Generation 2500 best: 57456 | Unique genomes: 76
Generation 3000 best: 57456 | Unique genomes: 75
Generation 3500 best: 57456 | Unique genomes: 82
Generation 4000 best: 57456 | Unique genomes: 79
Generation 4500 best: 57456 | Unique genomes: 80
Generation 4999 best: 57456 | Unique genomes: 86
------
Completed. Best route found:
('-22.974258000000003,-43.2011819', '-22.9687066,-43.1794453', '-22.95539,-43.167032', '-22.91314,-43.1799603', '-22.9119442,-43.196649', '-22.913406800000004,-43.2275963', '-22.9457624,-43.197282', '-22.9466961,-43.2177901', '-22.961238400000003,-43.2131231', '-22.9663481,-43.2200244', '-22.9710823,-43.2170445', '-23.0007675,-43.2727164', '-22.9966937,-43.2459426', '-22.9944913,-43.2348222', '-22.9871875,-43.2227147', '-22

**Generate the map visualization html file**

In [7]:
with open("visualize_map.template", "r") as templateFile:
    
    template = ''.join(templateFile.readlines())
    
    template = template.replace('<routelist>', '[\'' + '\',\''.join(result)+'\']')
    template = template.replace('<routemode>', directionsMode.upper())
    
    with open(outputHtmlFilePath, 'w') as htmlFile:
    
        htmlFile.write(template)
        
print('Map visualization html file generated successfully')

Map visualization html file generated successfully


**Opening the html file in a new tab**

In [8]:
import webbrowser
webbrowser.open_new_tab(outputHtmlFilePath)

print('Opened successfully')

Opened successfully
